In [1]:
cd ..

/Users/BEugeneSmith/Desktop/projects/PodcastAnalysis


In [66]:
# master dataset 
from Analysis.Helper import *
gen = datetime.now()
start = datetime(2017,7,24)
end = datetime(2019,8,21)

aai = AudioAnalysisInput(start=start,end=end)

filt_data = aai.data[(aai.data['datePlayed']>=start)&(~aai.data['batch_skip_ind'])]
filt_data = filt_data[['weekPlayed','album','duration',"dur_bin",'frequency','category']]
# filt_data['datePlayed'] = filt_data['datePlayed'].dt.date
filt_data['frequency_grp'] = filt_data['frequency'].apply(lambda x: "d" if x=="d" else "nd")
filt_data['duration'] = filt_data['duration']/3600

/Users/BEugeneSmith/Desktop/projects/PodcastAnalysis/Analysis/Helper.py:59: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(loaded_files).reset_index(drop=True),loaded_file_paths


In [67]:
filt_data.head()

,weekPlayed,album,duration,dur_bin,frequency,category,frequency_grp
18,2017_30,Global News Podcast,0.541576,03_long,d,news,d
19,2017_30,Linear Digressions,0.396111,02_medium,w,tech,nd
20,2017_30,FiveThirtyEight Politics,0.737500,03_long,w,politics,nd
21,2017_30,RISK!,0.911667,03_long,w,interview,nd
22,2017_30,Song of the Day,0.074202,01_short,d,music,d


In [68]:
filt_data = filt_data.replace("Make Me Smart® with Kai and Molly",'Make Me Smart with Kai and Molly')
filt_data = filt_data.replace("Slate's Political Gabfest",'Political Gabfest')
filt_data = filt_data.replace("Slate's The Gist",'The Gist')
filt_data = filt_data.replace("Marketplace with Kai Ryssdal",'Marketplace')
filt_data = filt_data.replace("Wait Wait... Don't Tell Me!","Wait Wait...Don't Tell Me!")


In [69]:
podcasts = filt_data['album'].unique()
durations = filt_data['dur_bin'].unique()
categories = filt_data['category'].unique()

In [70]:
data2 =filt_data.pivot_table(index=['album','category'],columns=["weekPlayed"],values=['frequency'],aggfunc="count")
stacked_data = data2.stack().reset_index()

In [71]:
last_weeks = filt_data['weekPlayed'].unique()[-4:]
last_weeks

array(['2019_31', '2019_32', '2019_33', '2019_34'], dtype=object)

In [72]:
weeks = []
for y in [2017,2018,2019]:
    for w in range(1,53):
        weeks.append("{}_{:02d}".format(y,w))
weeks[:5]

['2017_01', '2017_02', '2017_03', '2017_04', '2017_05']

In [73]:
catg_data = {
#     'category':[],
    'podcast':[],
    'week':[],
    'cont_code':[],
    'instance':[],
    'drop_off':[],
    'played_in_last_weeks':[]
}

for p in podcasts:
    cont_len = 1
    p_cast_data = stacked_data[stacked_data['album']==p]
    if len(p_cast_data) == 1:
#         catg_data['category'].append(c)
        dropoff_ind = (not(NextWeekList_Played_for_pcast in p_cast_data['weekPlayed'].unique())) or \
                (not(CurrWeekPlayed_for_pcast in last_weeks))
        catg_data['podcast'].append(p)
        catg_data['week'].append(p_cast_data.iloc[0]['weekPlayed'])
        catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
        catg_data['instance'].append("{:03d}".format(cont_len))
        catg_data['drop_off'].append(1 if not(p_cast_data.iloc[0]['weekPlayed'] in last_weeks) else 0)
        catg_data['played_in_last_weeks'].append(1 if p_cast_data.iloc[0]['weekPlayed'] in last_weeks else 0)
    else:
        for i in range(len(p_cast_data)):
            # get current 
            CurrWeekPlayed_for_pcast = p_cast_data.iloc[i]['weekPlayed']
            NextWeekList_Played_for_pcast = weeks[weeks.index(p_cast_data.iloc[i]['weekPlayed'])+1]
            
            # flags
            next_calcd_week_in_datelist = NextWeekList_Played_for_pcast in p_cast_data['weekPlayed'].unique()
            played_recently = CurrWeekPlayed_for_pcast in last_weeks
            dropoff_ind = (not(next_calcd_week_in_datelist)) and (not(played_recently))
            
            if p =="Global News Podcast":
                print(CurrWeekPlayed_for_pcast)
                print(NextWeekList_Played_for_pcast)
                print(dropoff_ind)
                print('====')
            if i == 0:
                catg_data['podcast'].append(p)
                catg_data['week'].append(CurrWeekPlayed_for_pcast)
                catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
                catg_data['instance'].append("{:03d}".format(cont_len))
                catg_data['drop_off'].append(1 if dropoff_ind else 0)
                catg_data['played_in_last_weeks'].append(1 if played_recently else 0)
                continue

            if next_calcd_week_in_datelist and not(played_recently):
                catg_data['podcast'].append(p)
                catg_data['week'].append(CurrWeekPlayed_for_pcast)
                catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
                catg_data['instance'].append("{:03d}".format(cont_len))
                catg_data['drop_off'].append(0)
                catg_data['played_in_last_weeks'].append(0)
                continue
                
            elif not(next_calcd_week_in_datelist) and (played_recently):
                catg_data['podcast'].append(p)
                catg_data['week'].append(CurrWeekPlayed_for_pcast)
                catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
                catg_data['instance'].append("{:03d}".format(cont_len))
                catg_data['drop_off'].append(1)
                catg_data['played_in_last_weeks'].append(1)
                cont_len+=1
                continue
            elif not(next_calcd_week_in_datelist) and not(played_recently):
                catg_data['podcast'].append(p)
                catg_data['week'].append(CurrWeekPlayed_for_pcast)
                catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
                catg_data['instance'].append("{:03d}".format(cont_len))
                catg_data['drop_off'].append(1) 
                catg_data['played_in_last_weeks'].append(0)
                cont_len+=1
            else:
                catg_data['podcast'].append(p)
                catg_data['week'].append(CurrWeekPlayed_for_pcast)
                catg_data['cont_code'].append("podcast_{}_{:03d}".format(p,cont_len))
                catg_data['instance'].append("{:03d}".format(cont_len))
                catg_data['drop_off'].append(0)
                catg_data['played_in_last_weeks'].append(1)
catg_data = pd.DataFrame(catg_data)

2017_30
2017_31
False
====
2017_31
2017_32
False
====
2017_32
2017_33
False
====
2017_33
2017_34
False
====
2017_34
2017_35
True
====
2017_51
2017_52
True
====


In [74]:
# catg_data.to_clipboard(index=False)
catg_data.head(10)

,cont_code,drop_off,instance,played_in_last_weeks,podcast,week
0,podcast_Global News Podcast_001,0,001,0,Global News Podcast,2017_30
1,podcast_Global News Podcast_001,0,001,0,Global News Podcast,2017_31
2,podcast_Global News Podcast_001,0,001,0,Global News Podcast,2017_32
3,podcast_Global News Podcast_001,0,001,0,Global News Podcast,2017_33
4,podcast_Global News Podcast_001,1,001,0,Global News Podcast,2017_34
5,podcast_Global News Podcast_002,1,002,0,Global News Podcast,2017_51
6,podcast_Linear Digressions_001,0,001,0,Linear Digressions,2017_30
7,podcast_Linear Digressions_001,0,001,0,Linear Digressions,2017_31
8,podcast_Linear Digressions_001,0,001,0,Linear Digressions,2017_32
9,podcast_Linear Digressions_001,0,001,0,Linear Digressions,2017_33


In [75]:
catg_data2_ages = catg_data.groupby(by=['podcast','cont_code'],).agg({'cont_code':'count','week':['min','max'],'drop_off':'sum','played_in_last_weeks':'sum'}).reset_index()
catg_data2_ages.columns = ["".join(x) for x in catg_data2_ages.columns]
catg_data2_ages = catg_data2_ages.groupby(by=['podcast']).agg({'cont_codecount':['mean','count','nunique','min','max'],'played_in_last_weekssum':'sum','weekmax':'max'}).reset_index()
catg_data2_ages.columns = ["".join(x) for x in catg_data2_ages.columns]
catg_data2_ages.to_clipboard(index=False)